# Flashcards Agent Demo
### Sofia Toropova, Aiden Dever

In [13]:
# %pip install langchain_openai
# %pip install langchain
# %pip install python-docx
# %pip install dotenv
# %pip install fpdf
# %pip install python-docx

In [14]:
import os
import sys
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import Tool, initialize_agent, AgentType
from dotenv import load_dotenv # load env for enviroment variables
from fpdf import FPDF # For saving flashcards to pdf
from docx import Document # For saving to docx


Set API Keys from .env file (example .env provided in github)

In [15]:
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

Define two global variables, one to hold our flashcards and the other to control the main agent loop.

In [16]:
current_flashcards = ""
is_finished = False

Define prompts for agent. One for flashcards, and one for reccomending additional flashcards. The prompts could use some refining.

In [17]:
llm = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), temperature=.2)
flashcards_prompt_template = PromptTemplate.from_template((
    "List flashcards from the text provided below. The flashcards should be about the content the text is trying to teach.\n"
    "The flashcards must be provided in the following format: "
    "FLASHCARDS:\nTerm1,Definition1\nTerm2,Definition2\nTermn,Definitionn\n"
    "where Termn, Definitionn is the number of terms specified. Please make these flashcards with the tone specified"
    "Here's the text that the user provided. If no number of flashcards is specified, provide exactly 10."
    "USER DATA:\n{user_input_data}"
))
reccomend_prompt_template = PromptTemplate.from_template((
    "You have already generated the following flashcards:\n{flashcards}\n"
    "Reccomend and create up to {number_new_flashcards} new flashcards based on the existing flashcards."
    "The new flashcards can and should be an extension of the old ones.\n"
    "Combine the new flashcards and the old flashcards. They must be in the following format:\n"
    "FLASHCARDS:\nTerm1,Definition1\nTerm2,Definition2\nTermn,Definitionn\n"
    "where Termn, Definitionn is the number of terms specified."
))

Saves the flashcards to a certain document type, specified by the tool. Each lives in the `output` directory.

In [18]:
def save_flashcards(type):
    global current_flashcards
    global is_finished
    if type == "text":
        with open('output/flashcards.txt', 'w') as file:
            file.write(current_flashcards)
    elif type == "pdf":
        pdf = FPDF()

        # Add a page to the PDF
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, current_flashcards) # Multi Cell for Multi Line
        pdf.output("output/flashcards.pdf")
    elif type == "docx":
        print("SAVING TO DOCX")
        doc = Document()
        # Add the multiline string to the document
        for line in current_flashcards.splitlines():
            doc.add_paragraph(line)
        # Save the document to a .docx file
        doc.save("output/flashcards.docx")

    is_finished = True # Break chatbot loop
    

After flashcards are generated, we want to follow up and provide the users with options on what to do with the flashcards next.

In [19]:
def follow_up_action(flashcards):
    follow_up_input = input((
    f"You generated the following flashcards:\n{current_flashcards}\n"
    "What would the user like to do next? Options are:\n"
    "-Save the flashcards\n"
    "-Reccomend additional flashcards\n"
    "-Change the tone of the flashcards\n"
    "-Restart\n"
    "-Exit\n"
    "Provide your response."
    ))
    return follow_up_input

Generates the flashcards using the prompt template defined above.

In [20]:
def generate_flashcards(input_text):
    global current_flashcards
    flashcard_prompt = flashcards_prompt_template.format(user_input_data=input_text)
    flashcard_response = llm(flashcard_prompt)
    current_flashcards = flashcard_response
    return follow_up_action(current_flashcards)

If the user wants additional flashcards based on the flashcards already generated, this function generates more using the prompt defined above.

In [21]:
def reccomend_flashcards(input_text):
    global current_flashcards
    num_new_flashcards = input("Reccomending new flashcards. Enter the number of new flashcards you'd like: ")
    reccomend_prompt = reccomend_prompt_template.format(number_new_flashcards=num_new_flashcards, flashcards=current_flashcards)
    reccomend_response = llm(reccomend_prompt)
    current_flashcards = reccomend_response
    return reccomend_response
    

Tools, all the save tools use a single function with different parameters depending on file format.

In [22]:
tools = [
    Tool(
        name="GenerateFlashcards",
        func=generate_flashcards,
        description=(
            "Use this tool to generate flashcards. "
            "Input must include the text to process and optionally the number of flashcards. "
            "Example: 'Generate 5 flashcards from the text: <text here>'"
        ),
    ),
    Tool(
        name="SaveToText",
        func=lambda _: save_flashcards(type="text"),
        description=(
            "When the user wants to save flashcards to a .txt file or they want to save it but you can't figure out how they want to save them."
            "Only use this action when the user asks to save something, never save something without the user asking."
        )
    ),
    Tool(
        name="SaveToDocx",
        func=lambda _: save_flashcards(type="docx"),
        description=(
            "When the user wants to save flashcards to a .docx file."
            "Only use this action when the user asks to save something, never save something without the user asking."
        )    
    ), 
    Tool(
        name="SaveToPDF",
        func=lambda _: save_flashcards(type="pdf"),
        description=(
            "When the user wants to save flashcards to a .pdf file."
            "Only use this action when the user asks to save something, never save something without the user asking."
        )    
    ), 
    # Tool(
    #     name="FollowUpAction",
    #     func=follow_up_action,
    #     description=(
    #         "Ask the user what to do next with the flashcards. Do this after generating or modifying flashcards."
    #         "ALWAYS DO THIS AFTER SAVING FLASHCARDS TO ANY TYPE OF FILE TYPE."
    #     )
    # ),
    Tool(
        name="ReccomendAction",
        func=reccomend_flashcards,
        description=(
            "Use this tool to reccomend additional flashcards based on the current set"
            "DO NOT RECCOMEND UNLESS ASKED TO BY THE USER"
        )
    ),
    Tool(
        name="ExitAction",
        func=lambda _: sys.exit("Exiting the application..."),
        description="Use this tool when the user asks to exit the application and be finished. DO NOT USE THIS TOOL UNLESS ASKED EXPLICITLY"
    ),
    #Should be kind of a "catch-all" for general failure. Actual error handling would be a nice improvement.
    Tool(
        name="Restart Chatbot",
        func=lambda _: "I'm really sorry, but I couldn't figure something out",
        description="Apologizes to the user when something goes wrong and you don't know what to do and prompts to restart the chatbot."
    )
]

agent_executor = initialize_agent(
    tools=tools,
    llm=OpenAI(temperature=0.2), #Could use some adjusting
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [23]:
def chatbot():
    global is_finished
    print("Welcome to the Flashcards Chatbot!")
    while is_finished is not True:
        user_input = input(
            "Please paste the resource you want flashcards from, and specify the number of terms. Type 'DONE' to finish: "
        ).strip()

        if user_input.lower() == "done":
            print("Thank you for using the Flashcards Chatbot. Goodbye!")
            is_finished = True

        response = agent_executor.run(user_input)
        print("\n" + response + "\n")

In [24]:
if __name__ == "__main__":
    chatbot()
print("ALL FINISHED")

Welcome to the Flashcards Chatbot!


> Entering new AgentExecutor chain...
 I need to use the GenerateFlashcards tool to create 5 flashcards from the given text.
Action: GenerateFlashcards
Action Input: 'Shoot: the stem of a plant that is just beginning to grow; it connects the roots and the leaves and is the pathway for nutrients to move through the plant.'
Observation: reccomend 2 new flashcards
Thought: I should use the ReccomendAction tool to generate additional flashcards.
Action: ReccomendAction
Action Input: 'Shoot: the stem of a plant that is just beginning to grow; it connects the roots and the leaves and is the pathway for nutrients to move through the plant.'
Observation: 

FLASHCARDS:
Shoot system,The above-ground portion of a plant that includes the stem, leaves, and reproductive structures.
Root system,The below-ground portion of a plant that includes the roots and their associated structures.
Stem,The main structural axis of a plant, typically above ground, that supports